<h4>Réalisation d’un mini solveur de CSP binaires
Master MPRO zribi-parmentier</h4>

In [447]:
# -*- coding: utf-8 -*-
import time
import timeit

variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
instance=dict()
instance={}
#instance["x1"]=2
#instance["x2"]=1
#0: faux, 1: vérifié 2: incomplet


In [448]:
print(2==variables['x1'])
print(variables.keys())

False
dict_keys(['x1', 'x2'])


In [449]:
print('x1' in variables.keys())

True


In [450]:
print(contraintes)
print(contraintes[0][1][0])

[[('x1', 'x2'), [(1, 2), (2, 2)]]]
(1, 2)


In [451]:
print(contraintes[0])

[('x1', 'x2'), [(1, 2), (2, 2)]]


In [452]:
"""
variables["x1"]=1
variables["x2"]=2
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
"""
#checkconstraint sur une instance partielle
#Instance partielle: couple de valeurs pour x1, x2.
#{'x1':1,'x2':2}
#Pour tout checker, for loop sur les instances partielles, contraintes.
# ici constraint est de la forme [('x1', 'x2'), [(1, 2), (2, 2)]
# par ex contraintes[0]
def checkConstraint(constraint,instance):
    var1=constraint[0][0]
    var2=constraint[0][1]
    #print('var1 check: ',var1)
    #print('var2 check: ',var2)
    #print('instance keys ', instance.keys())
    if var1 in instance.keys() and var2 in instance.keys():
        for j in constraint[1]:
            if j[0]==instance[var1] and j[1]==instance[var2]:
                return(1)
        return(0)
    else:
        return(2)
        

print(checkConstraint(contraintes[0],{'x1':1,'x2':2}))

1


In [453]:
def strategy_selection_1(variables,instance):
     for x in variables.keys():
        if not(x in instance.keys()):
            return x
        
def selectVarAj(variables,instance):
    return strategy_selection_1(variables,instance)

In [454]:
print("=====================")
solutions=[]
def backTrack(instance,contraintes,n):
    complet=True
    for c in contraintes:
        ch=checkConstraint(c,instance)
        if ch==0:
            print(ch,False,"instance fausse !!!")
            return(False)
        if ch==2:
            print("instance incomplete")
            complet=False
    if complet==True:
        print("instance complete")
        print("resultat:",instance)
        solutions.append(instance)
        return(True)
    #On choisit une variable non instancée
    varAj=selectVarAj(variables,instance)
    #reordonner eventuellement varAj avant
    #Pour chaque valeur y de varAj dans son domaine
    for y in variables[varAj]:
        print(varAj,"varAj ")
        print(y,"y")
        #On ajoute les valeurs possibles de varAj
        instance[varAj]=y
        print("instance actuelle",instance)
        if backTrack(instance,contraintes,n):
            #quand ca retourne faux, ca tente une autre valeur de varj
            print("************")
            #return True
    return(False)

In [455]:
instance=dict()
variables["x1"]=[1,2]
variables["x2"]=[1,2]
print(backTrack(instance,contraintes,0))

#print(instance,"instance1")

#print(variables["x1"])

instance incomplete
x1 varAj 
1 y
instance actuelle {'x1': 1}
instance incomplete
x2 varAj 
1 y
instance actuelle {'x1': 1, 'x2': 1}
0 False instance fausse !!!
x2 varAj 
2 y
instance actuelle {'x1': 1, 'x2': 2}
instance complete
resultat: {'x1': 1, 'x2': 2}
************
x1 varAj 
2 y
instance actuelle {'x1': 2, 'x2': 2}
instance complete
resultat: {'x1': 2, 'x2': 2}
************
False


In [456]:
instance

{'x1': 2, 'x2': 2}

In [457]:
contraintes

[[('x1', 'x2'), [(1, 2), (2, 2)]]]

In [458]:
#Fonction séparée pour la selection du couple dans l'algo AC_3.
#Peut etre utile si existence d heuristiques interessantes
def selectCouple(aTester):
    #Different strategies can be chosen for the selection of the couple
    return aTester.pop()

In [459]:
a=[3,4,5]
selectCouple(a)
print(a)

[3, 4]


In [460]:
""""Fonction utilisee dans l'algo AC-3
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes"""
def checkConstraint_ac3(constraints,instance):
    var1=list(instance.keys())[0]
    var2=list(instance.keys())[1]
    for c in constraints:
        if var1 in c[0] and var2 in c[0]:
            #si les nombres n'apparaissent meme pas dans les possibilites, on passe
            # a l iteration suivante
            """for i in range(len(c[1])):
                #check
                print("yes")
                print(var1,var2)
                print(c[1][i])
                print("-----")
                if instance[var1]==c[1][i][0] and instance[var2]==c[1][i][1]:
                    return True
                """
            if (instance[var1],instance[var2]) in c[1]:
                return True
    return False

Remarque: nouveau code ac_3, semble bien fonctionner. Testé sur l exemple du cours sur la coloration de graphe.

In [461]:
"""
Algorithme AC-3
quand on supprime une valeur a la variable x,
on ne reexamine que les variables liees a x par une contrainte"""
def ac_3(variables,constraints):
    aTester=[]
    """Initialisation aTester"""
    for i in range(len(constraints)):
       # print('constraints[i]:',constraints[i])
        #on ajoute le couple (x,y)
        aTester.append(constraints[i][0])
        #print('Atester:',aTester)
    while len(aTester)!=0:
        """"Selection du couple """
        couple=selectCouple(aTester)
        list_values_without_support=[]
        #print('-*-*-*-*-*-*-')
        #print('new couple')
        #print('list no support',list_values_without_support)
        #pr  chaque valeur pour la variable x
        #print('couple:',couple)
        """"Parcourts des valeurs de x """
        for value_x in variables[couple[0]]:
            #print('nouvelle valeur de x',couple[0],value_x)
            support=False
            """"Parcourt des valeurs de y """
            for value_y in variables[couple[1]]:
                """"On verifie qu'il existe une contrainte Cx,y validee"""
                if checkConstraint_ac3(contraintes,{couple[0]:value_x,couple[1]:value_y}):
                    support=True
                    break;
            #print('support',support,couple[0],value_x,couple[1])
            """"On cree la liste des valeurs de x sans support"""
            if not support:
                #print('x', couple[0])
                list_values_without_support.append(value_x)
                #print('list no support',list_values_without_support)

        if len(list_values_without_support)!=0:
            #print('===')
            #print('reduce domaine of', couple[0], list_values_without_support)
            """"On met a jour le domaine de x """
            variables[couple[0]]=list(set(variables[couple[0]])-set(list_values_without_support))
            #print('variables: ',variables)
            """"On met a jour aTester"""
            for i in range(len(constraints)):
                if constraints[i][0][0]!=couple[1] and constraints[i][0][1]==couple[0]:
                    #print('couple étudié ', couple)
                    #print('ajout dans aTester',constraints[i][0])
                    aTester.append([constraints[i][0][0],constraints[i][0][1]])
                    #print('======')
            #print('======')
        #print('-*-*-*-*-*-*-')
       
                    
                                
    return True
            

In [462]:
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]]
            ]
t1 = time.time()
ac_3(variables,contraintes)
t2=time.time()
duree=t2-t1

In [463]:
variables

{'x1': [3], 'x2': [2], 'x3': [1]}

In [464]:
duree

0.00011301040649414062

In [465]:
""""Fonction utilisee dans l'algo AC_3_Bis
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes"""
def checkConstraint_ac3_bis(variables, constraints,var1,value_var1,var2):
    for c in constraints:
        if var1 in c[0] and var2 in c[0]:
            for value_var2 in variables[var2]:
                if (value_var1,value_var2) in c[1]:
                    return True
    return False

In [466]:
"""
Algorithme AC-3
quand on supprime une valeur a la variable x,
on ne reexamine que les variables liees a x par une contrainte"""
"""" Difference avec ac_3 ? au niveau de la fonction checkConstraint_ac3, ordre for c in constraints
et for value_y in domaines de y"""
""""Performances ac_3_bis peut etre legerement meilleur que ac_3"""
def ac_3_bis(variables,constraints):
    aTester=[]
    """Initialisation aTester"""
    for i in range(len(constraints)):
       # print('constraints[i]:',constraints[i])
        #on ajoute le couple (x,y)
        aTester.append(constraints[i][0])
    while len(aTester)!=0:
        """"Selection du couple """
        couple=selectCouple(aTester)
        list_values_without_support=[]
        #print('-*-*-*-*-*-*-')
        #print('new couple')
        #print('list no support',list_values_without_support)
        #pr  chaque valeur pour la variable x
        #print('couple:',couple)
        """"Parcourts des valeurs de x """
        for value_x in variables[couple[0]]:
            #print('nouvelle valeur de x',couple[0],value_x)
            support=False
            """"On verifie qu'il existe une contrainte Cx,y validee"""
            if checkConstraint_ac3_bis(variables,constraints,couple[0],value_x,couple[1]):
                support=True
                break;
            #print('support',support,couple[0],value_x,couple[1])
            """"On cree la liste des valeurs de x sans support"""
            if not support:
                #print('x', couple[0])
                list_values_without_support.append(value_x)
                #print('list no support',list_values_without_support)

        if len(list_values_without_support)!=0:
            #print('===')
            #print('reduce domaine of', couple[0], list_values_without_support)
            """"On met a jour le domaine de x """
            variables[couple[0]]=list(set(variables[couple[0]])-set(list_values_without_support))
            #print('variables: ',variables)
            """"On met a jour aTester"""
            for i in range(len(constraints)):
                if constraints[i][0][0]!=couple[1] and constraints[i][0][1]==couple[0]:
                    #print('couple étudié ', couple)
                    #print('ajout dans aTester',constraints[i][0])
                    aTester.append([constraints[i][0][0],constraints[i][0][1]])
                    #print('======')
            #print('======')
        #print('-*-*-*-*-*-*-')
       
                    
                                
    return True
            

In [467]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]]
            ]
t1 = time.time()
ac_3_bis(variables,contraintes)
t2=time.time()
duree=t2-t1


In [468]:
variables

{'x1': [3], 'x2': [2], 'x3': [1]}

In [469]:
duree

0.00011992454528808594

In [474]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]
t1 = time.time()
ac_3_bis(variables,contraintes)
t2=time.time()
duree=t2-t1

In [475]:
variables

{'x1': [3], 'x2': [2], 'x3': [1], 'x4': [2]}

In [476]:
duree

0.0002727508544921875

In [477]:
"""Forward Checking
idéé: anticiper certaines prises de decisions dans le backtrack.
Dès qu’une variable x est instanciee à la valeur a, 
on filtre toutes les valeurs incompatibles avec〈x,a〉mais on ne propage pas plus.
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes
var1 est la variable que l'on analyse.
"""
def forwardChecking(instance_partielle,variables,constraints,var1):
    for var2 in variables.keys():
        list_no_support=[]
        if var2 not in instance_partielle.keys():
            exist_constraint=False
            for c in constraints:
                if (var1,var2)==c[0]:
                    exist_constraint=True
                    break
            if exist_constraint:
                for value_var2 in variables[var2]:
                    #for loop supplementaire dans le cas où les contraintes sur x1,x2 ne sont 
                    #pas regroupees dans un seul bloc de constraints. A modifier si c est le cas
                    #finalement
                    for c in constraints:
                        if (var1,var2)==c[0] and not (instance_partielle[var1],value_var2) in c[1]:
                            list_no_support.append(value_var2)
        if len(list_no_support)!=0:
            variables[var2]=list(set(variables[var2])-set(list_no_support))
    return True


In [478]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
variables["x3"]=[1,3]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]

In [479]:
instance_partielle={"x1":3}

In [480]:
forwardChecking(instance_partielle,variables,contraintes,"x1")

True

In [481]:
variables

{'x1': [1, 2], 'x2': [1, 2], 'x3': [1], 'x4': [1, 2]}